In [ ]:
Query FindOrd() As websys.Query(ROWSPEC = "name:%String:姓名,no:%String:登记号,idno:%String:身份证号,phone:%String:联系方式,admDate:%String:就诊日期,prescDate:%String:处方日期,loc:%String:科室,bill:%String:费别,prescNo:%String:处方号,sum:%String:处方总金额") [ SqlProc ]
{
}

/// debug:	d ##class(%ResultSet).RunQuery("web.DHCDocMain", "FindOrd")
/// select * from web.DHCDocMain_FindOrd()
ClassMethod FindOrdExecute(ByRef qHandle As %Binary) As %Status
{
	s $zt="muse"
	s repid = $i(^CacheTemp)
	s ind = 1
	s qHandle = $lb(0, repid, 0)
	;s languageId = ..%LanguageID()
	
	s sum = 0
	s pc = 0
	s oc = 0
	s no = 0	
	for {
		s no = $o(^OEORD(0,"PrescNo",no))
		q:no=""
		s ord = 0
		s solo = 0
		s psum = 0
		for {
			s ord = $o(^OEORD(0,"PrescNo",no,ord))
			q:ord=""
			s item = 0
			for {
				s item = $o(^OEORD(0,"PrescNo",no,ord,item))
				q:item=""
				; 自费
				; 开始日期 开医嘱日期
				s stt = $p($g(^OEORD(ord,"I",item,1)),"^",9)
				s date = $p($g(^OEORD(ord,"I",item,3)),"^",7)
				;b ;;; stt
				;continue:stt>=date
				s EpisodeId = +^OEORD(ord)
				s type = $p(^PAADM(EpisodeId),"^",2)
				continue:type'="O"
				s adm = $p(^PAADM(EpisodeId),"^",6)
				continue:stt>=adm
				s format = $zd(stt,3)
				continue:format'["2024"
				s reason = $p($g(^OEORD(ord,"I",item,11)),"^",18)
				continue:reason=1
				
				s stat = ##class(web.DHCDocQryOEOrder).GetdsStatusDate(ord_"||"_item)
				continue:stat'="已发"
				; 姓名、登记号、身份证号、联系方式、科室、费别、处方号、处方总金额
				s pat = $p(^PAADM(EpisodeId),"^",1)
				s name = $p(^PAPER(pat,"ALL"),"^")
				s num = $p(^PAPER(pat,"PAT",1),"^")
				s idno = $p(^PAPER(pat,"ALL"),"^",9)
				s phone = $p(^PAPER(pat,"ALL"),"^",4)
				s bill = $p(^PAC("ADMREA",reason),"^",2)
				s loc = $p($g(^OEORD(ord,"I",item,1)),"^",3)
				s loc = $p($g(^CTLOC(loc)),"^",2)
				s:phone="" phone = $p(^PAPER(pat,"PER",1),"^",11)
				s:phone="" phone = $p(^PAPER(pat,"PER",4),"^",21)
				s arcim = $p($g(^OEORD(ord,"I",item,3)),"^",2)
				s desc = $p($g(^ARCIM(+arcim,1,1)),"^",2)
				s oc = oc + 1
				s solo = solo + 1
				s adm = $zd(adm,3)
				s stt = $zd(stt,3)
				s ArcPrice=##class(web.DHCDocOrderCommon).GetOEORIPrice(ord_"||"_item)
				s ArcPrice=$fn($p(ArcPrice,"^",1),"",4)
		
				s BillingUOMDR=##Class(web.DHCDocOrderCommon).GetOrdPackUOMDR(ord_"||"_item)
			
		
				s ProtocolPackUOMDR=$p($g(^OEORD(ord,"I",item,"DHC")),"^",13)
				s:BillingUOMDR="" BillingUOMDR=ProtocolPackUOMDR
				s convFac=##class(appcom.OEDispensing).convFac(arcim,BillingUOMDR)
				s OrdPackQtyInfo=##class(web.DHCDocQryOEOrder).GetOrdPackQtyInfo(ord_"||"_item)
				
				s OrderPackQty=$p(OrdPackQtyInfo,"^",1)
				s PackUOMDr=$p(OrdPackQtyInfo,"^",3)
				s QtyPackUOM=""
				;s:(OrderPackQty'="") QtyPackUOM=OrderPackQty_PackUOMDesc //$p(OrdPackQtyInfo,"^",3)
				s DspConfirmQty=""
				s ActiveQty=$p(OrdPackQtyInfo,"^",7)
				s Pqty= ActiveQty
				if (+ActiveQty=0){
					s BaseQty=$p(OrdPackQtyInfo,"^",4) //换算成基本单位后的数量
					s OrderSum=(BaseQty/convFac)*ArcPrice
					s OrderSum=$j(OrderSum,"",2)
				}else{
					s OrderSum=(ActiveQty/convFac)*ArcPrice
					//s QtyPackUOM=(ActiveQty/convFac)_PackUOMDesc
					s OrderSum=$j(OrderSum,"",2)
					
				}
				s sum = sum + $g(OrderSum)
				s psum = psum + $g(OrderSum)
				s id = oc
			}
		}
		s:solo'=0 pc = pc + 1
		if solo'=0 {
			; 姓名、登记号、身份证号、联系方式、科室、费别、处方号、处方总金额
			d OutputRow
		}
	}
	s name = "合计"
	
	s num = pc
	s idno = oc
	s phone=""
	s loc = ""
	s bill = ""
	s no = ""
	s psum = sum
	s adm = ""
	s stt = ""
	d OutputRow
	Quit $$$OK

OutputRow
	s Data = $lb($g(name),$g(num),$g(idno),$g(phone),$g(adm),$g(stt),$g(loc),$g(bill),$g(no),$g(psum))
	s ^CacheTemp(repid, ind) = Data
	s ind = ind + 1
	q
muse
	s $zt=""
	s ^muse=$lb($ze,$zdt($h,3))
	q $ze
}
